In [44]:
from prep_data import Data_prepare
import numpy as np 
import os 
# import prep_data

###  Data preprocessing 
1. covert mat data to numpy array 
2. combine male and female data to one array 
3. if needed, remove certain rows 
4. get postive and negative of Xs 
    - order difference or net vs neg 
5. reshape each X: 128*8*2 = 2048 
    - input: data 
6. generate labels for x 
    - input: data and label_type (0:zeros, 1: ones )
7. combine net and neg 
    - input: netX,netY,negX,negY

In [45]:
dp = Data_prepare() 
pos = [[0,3],[0,2],[0,1],[0,0]]
neg = [[1,3],[1,2],[1,1],[1,0]]
# netural vs postive 
# pos = [[0,0],[0,1],[1,2],[1,3],[2,0],[2,1],[2,2],[2,3]]
# neg = [[0,2],[0,3],[1,0],[1,1],[3,0],[3,1],[3,2],[3,3]] 
m_file = "./raw_data/Emotrans1_Boy_data_preprocessed_42.mat" 
f_file = "./raw_data/Emotrans1_girl_data_preprocessed_42.mat"
# index = [11,18,36]
df = dp.combine_male_female(m_file,f_file)
df = dp.remove_person(df,index)
pos_df = dp.get_pos_or_neg(df,pos)
neg_df = dp.get_pos_or_neg(df,neg)
pos_sequeezed = dp.squeeze_feature_size(pos_df)
neg_sequeezed = dp.squeeze_feature_size(neg_df)
pos_labels = dp.generate_labels(pos_sequeezed,1)
neg_labels = dp.generate_labels(neg_sequeezed,0)
x,y = dp.combine_net_neg(pos_sequeezed,pos_labels,neg_sequeezed,neg_labels)

#### Normalize x with standardScaler

In [46]:
from sklearn.preprocessing import StandardScaler
def normalize(train,test):
    new_x = np.append(train,test,axis=0)
    scaler = StandardScaler()
    scaler.fit(new_x)
    train = scaler.transform(train)
    test= scaler.transform(test)
    return train, test

In [47]:
## reshape x_train and x_test to 3D for LSTM 
def reshape_x(x_train,x_test,feature_num):
    x_train = np.reshape(x_train, (x_train.shape[0],1,feature_num))
    x_test = np.reshape(x_test, (x_test.shape[0],1,feature_num))
    return x_train,x_test

In [48]:
def get_x_y_train(x,y):
    new_x = x[0]
    new_y = y[0]
    for i in range(1,len(x)):
        # print(new_x.shape)
        new_x=np.append(new_x,x[i],axis=0)
        new_y=np.append(new_y,y[i],axis=0)
    return new_x,new_y

In [49]:
# y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
# y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

In [50]:
temp1, temp2 = get_x_y_train(x,y)
print(temp1.shape,temp2.shape)

(1374, 2048) (1374, 1)


#### LSTM model 

In [51]:
import tensorflow as tf
from tensorflow.keras import Sequential

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from keras.optimizers import SGD

def lstm_model():
    tf.keras.backend.clear_session()

    model = Sequential()
    model.add(LSTM(512, input_shape=(1,2048),activation="relu",return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(256,activation="relu",return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(128,activation="relu",return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(128,activation="relu",return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(16,activation="relu"))
    model.add(Dropout(0.2))
    # model.add(LSTM(50))
    # model.add(Dropout(0.2))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
    return model 

In [52]:
## leave one out 
# print(new_x.shape,new_y.shape)
all_accuracy=[]
for d in range(x.shape[0]):
    print("leave one out # person:",d)
    if d == 0:
        x_train,y_train = get_x_y_train(x[d+1:],y[d+1:])
        x_test,y_test =x[d],y[d]
    elif d == x.shape[0]-1:
        x_train,y_train = get_x_y_train(x[:d],y[:d])
        x_test,y_test = x[d],y[d]
    else:
        x_train,y_train = get_x_y_train(np.append(x[:d],x[d+1:],axis=0),np.append(y[:d],y[d+1:],axis=0))
        x_test,y_test = x[d],y[d]
    # scale 
    x_train,x_test = normalize(x_train,x_test)
    # reshape  
    x_train,x_test = reshape_x(x_train,x_test,2048)
    # define model 
    model = lstm_model()
    # train 
    history = model.fit(x_train, y_train, epochs = 50, validation_data= (x_test, y_test))
    score, acc = model.evaluate(x_test, y_test)
    all_accuracy.append(acc)

leave one out # person: 0
Epoch 1/50
42/42 [==============================] - 2s 36ms/step - loss: 0.6991 - accuracy: 0.5538 - val_loss: 0.5184 - val_accuracy: 0.8333
Epoch 2/50
42/42 [==============================] - 1s 22ms/step - loss: 0.6491 - accuracy: 0.7040 - val_loss: 0.3517 - val_accuracy: 0.8889
Epoch 3/50
42/42 [==============================] - 1s 21ms/step - loss: 0.5933 - accuracy: 0.7354 - val_loss: 0.3600 - val_accuracy: 0.8889
Epoch 4/50
42/42 [==============================] - 1s 22ms/step - loss: 0.5553 - accuracy: 0.7519 - val_loss: 0.2370 - val_accuracy: 0.8889
Epoch 5/50
42/42 [==============================] - 1s 21ms/step - loss: 0.4967 - accuracy: 0.7773 - val_loss: 0.2672 - val_accuracy: 0.9167
Epoch 6/50
42/42 [==============================] - 1s 22ms/step - loss: 0.4738 - accuracy: 0.7937 - val_loss: 0.2323 - val_accuracy: 0.9167
Epoch 7/50
42/42 [==============================] - 1s 22ms/step - loss: 0.4172 - accuracy: 0.8206 - val_loss: 0.2399 - val_accu

Epoch 8/50
42/42 [==============================] - 1s 22ms/step - loss: 0.4437 - accuracy: 0.8117 - val_loss: 0.2663 - val_accuracy: 0.8611
Epoch 9/50
42/42 [==============================] - 1s 22ms/step - loss: 0.4149 - accuracy: 0.8296 - val_loss: 0.3625 - val_accuracy: 0.7500
Epoch 10/50
42/42 [==============================] - 1s 22ms/step - loss: 0.3576 - accuracy: 0.8423 - val_loss: 0.3264 - val_accuracy: 0.8056
Epoch 11/50
42/42 [==============================] - 1s 22ms/step - loss: 0.3962 - accuracy: 0.8520 - val_loss: 0.3036 - val_accuracy: 0.8333
Epoch 12/50
42/42 [==============================] - 1s 22ms/step - loss: 0.3562 - accuracy: 0.8692 - val_loss: 0.3406 - val_accuracy: 0.8056
Epoch 13/50
42/42 [==============================] - 1s 21ms/step - loss: 0.2710 - accuracy: 0.8819 - val_loss: 0.4305 - val_accuracy: 0.8056
Epoch 14/50
42/42 [==============================] - 1s 21ms/step - loss: 0.2631 - accuracy: 0.8946 - val_loss: 0.4152 - val_accuracy: 0.8056
Epoch 15

Epoch 15/50
42/42 [==============================] - 1s 21ms/step - loss: 0.2622 - accuracy: 0.8834 - val_loss: 0.6990 - val_accuracy: 0.7222
Epoch 16/50
42/42 [==============================] - 1s 21ms/step - loss: 0.1988 - accuracy: 0.9111 - val_loss: 0.8013 - val_accuracy: 0.7500
Epoch 17/50
42/42 [==============================] - 1s 20ms/step - loss: 0.1878 - accuracy: 0.9268 - val_loss: 0.9166 - val_accuracy: 0.8333
Epoch 18/50
42/42 [==============================] - 1s 20ms/step - loss: 0.2698 - accuracy: 0.9305 - val_loss: 0.6725 - val_accuracy: 0.8056
Epoch 19/50
42/42 [==============================] - 1s 20ms/step - loss: 0.1935 - accuracy: 0.9312 - val_loss: 0.6803 - val_accuracy: 0.8611
Epoch 20/50
42/42 [==============================] - 1s 21ms/step - loss: 0.1866 - accuracy: 0.9297 - val_loss: 0.8533 - val_accuracy: 0.7778
Epoch 21/50
42/42 [==============================] - 1s 20ms/step - loss: 0.1702 - accuracy: 0.9350 - val_loss: 1.5719 - val_accuracy: 0.7222
Epoch 

Epoch 22/50
42/42 [==============================] - 1s 21ms/step - loss: 0.2810 - accuracy: 0.9215 - val_loss: 1.8695 - val_accuracy: 0.5278
Epoch 23/50
42/42 [==============================] - 1s 21ms/step - loss: 0.1411 - accuracy: 0.9514 - val_loss: 3.4099 - val_accuracy: 0.5556
Epoch 24/50
42/42 [==============================] - 1s 20ms/step - loss: 0.1991 - accuracy: 0.9410 - val_loss: 2.6553 - val_accuracy: 0.5278
Epoch 25/50
42/42 [==============================] - 1s 20ms/step - loss: 0.1252 - accuracy: 0.9559 - val_loss: 3.5757 - val_accuracy: 0.5556
Epoch 26/50
42/42 [==============================] - 1s 20ms/step - loss: 0.1442 - accuracy: 0.9507 - val_loss: 3.5418 - val_accuracy: 0.5556
Epoch 27/50
42/42 [==============================] - 1s 21ms/step - loss: 0.1288 - accuracy: 0.9619 - val_loss: 3.8479 - val_accuracy: 0.6389
Epoch 28/50
42/42 [==============================] - 1s 20ms/step - loss: 0.2050 - accuracy: 0.9574 - val_loss: 2.5582 - val_accuracy: 0.5556
Epoch 

Epoch 29/50
42/42 [==============================] - 1s 21ms/step - loss: 0.1241 - accuracy: 0.9641 - val_loss: 0.1733 - val_accuracy: 0.9167
Epoch 30/50
42/42 [==============================] - 1s 21ms/step - loss: 0.1370 - accuracy: 0.9596 - val_loss: 0.6890 - val_accuracy: 0.9167
Epoch 31/50
42/42 [==============================] - 1s 21ms/step - loss: 0.1141 - accuracy: 0.9753 - val_loss: 0.1399 - val_accuracy: 0.8889
Epoch 32/50
42/42 [==============================] - 1s 21ms/step - loss: 0.0834 - accuracy: 0.9686 - val_loss: 0.5046 - val_accuracy: 0.8611
Epoch 33/50
42/42 [==============================] - 1s 21ms/step - loss: 0.0974 - accuracy: 0.9723 - val_loss: 0.8627 - val_accuracy: 0.8611
Epoch 34/50
42/42 [==============================] - 1s 21ms/step - loss: 0.1091 - accuracy: 0.9649 - val_loss: 1.2707 - val_accuracy: 0.7500
Epoch 35/50
42/42 [==============================] - 1s 21ms/step - loss: 0.1101 - accuracy: 0.9716 - val_loss: 0.5547 - val_accuracy: 0.8889
Epoch 

...

42/42 [==============================] - 1s 21ms/step - loss: 0.4478 - accuracy: 0.8132 - val_loss: 0.8668 - val_accuracy: 0.6944
Epoch 8/50
42/42 [==============================] - 1s 21ms/step - loss: 0.3911 - accuracy: 0.8333 - val_loss: 1.2865 - val_accuracy: 0.6944
Epoch 9/50
42/42 [==============================] - 1s 21ms/step - loss: 0.3486 - accuracy: 0.8640 - val_loss: 1.3899 - val_accuracy: 0.7222
Epoch 10/50
42/42 [==============================] - 1s 21ms/step - loss: 0.3218 - accuracy: 0.8737 - val_loss: 2.2965 - val_accuracy: 0.7222
Epoch 11/50
42/42 [==============================] - 1s 21ms/step - loss: 0.4834 - accuracy: 0.8767 - val_loss: 1.1810 - val_accuracy: 0.6111
Epoch 12/50
42/42 [==============================] - 1s 21ms/step - loss: 0.3137 - accuracy: 0.8886 - val_loss: 2.6508 - val_accuracy: 0.6944
Epoch 13/50
42/42 [==============================] - 1s 21ms/step - loss: 0.4084 - accuracy: 0.9111 - val_loss: 2.0301 - val_accuracy: 0.6667
Epoch 14/50
42/42 [=

42/42 [==============================] - 1s 21ms/step - loss: 0.2765 - accuracy: 0.8924 - val_loss: 0.5953 - val_accuracy: 0.7500
Epoch 15/50
42/42 [==============================] - 1s 21ms/step - loss: 0.2836 - accuracy: 0.9028 - val_loss: 1.2359 - val_accuracy: 0.8056
Epoch 16/50
42/42 [==============================] - 1s 21ms/step - loss: 0.1945 - accuracy: 0.9141 - val_loss: 2.2826 - val_accuracy: 0.8333
Epoch 17/50
42/42 [==============================] - 1s 21ms/step - loss: 0.3092 - accuracy: 0.9253 - val_loss: 0.7017 - val_accuracy: 0.8333
Epoch 18/50
42/42 [==============================] - 1s 21ms/step - loss: 0.2393 - accuracy: 0.9410 - val_loss: 1.2773 - val_accuracy: 0.7500
Epoch 19/50
42/42 [==============================] - 1s 21ms/step - loss: 0.2216 - accuracy: 0.9297 - val_loss: 2.2123 - val_accuracy: 0.8333
Epoch 20/50
42/42 [==============================] - 1s 21ms/step - loss: 0.3323 - accuracy: 0.9335 - val_loss: 1.0599 - val_accuracy: 0.8611
Epoch 21/50
42/42 

42/42 [==============================] - 1s 19ms/step - loss: 0.1864 - accuracy: 0.9487 - val_loss: 0.5314 - val_accuracy: 0.7667
Epoch 22/50
42/42 [==============================] - 1s 19ms/step - loss: 0.1750 - accuracy: 0.9539 - val_loss: 1.1214 - val_accuracy: 0.7667
Epoch 23/50
42/42 [==============================] - 1s 19ms/step - loss: 0.1936 - accuracy: 0.9509 - val_loss: 0.4006 - val_accuracy: 0.8667
Epoch 24/50
42/42 [==============================] - 1s 19ms/step - loss: 0.1478 - accuracy: 0.9583 - val_loss: 1.4013 - val_accuracy: 0.8000
Epoch 25/50
42/42 [==============================] - 1s 19ms/step - loss: 0.1662 - accuracy: 0.9568 - val_loss: 0.3665 - val_accuracy: 0.9000
Epoch 26/50
42/42 [==============================] - 1s 19ms/step - loss: 0.1421 - accuracy: 0.9621 - val_loss: 0.7033 - val_accuracy: 0.8333
Epoch 27/50
42/42 [==============================] - 1s 19ms/step - loss: 0.0692 - accuracy: 0.9799 - val_loss: 0.9307 - val_accuracy: 0.8333
Epoch 28/50
42/42 

42/42 [==============================] - 1s 19ms/step - loss: 0.1558 - accuracy: 0.9516 - val_loss: 1.6856 - val_accuracy: 0.6333
Epoch 29/50
42/42 [==============================] - 1s 19ms/step - loss: 0.1720 - accuracy: 0.9613 - val_loss: 1.1870 - val_accuracy: 0.6333
Epoch 30/50
42/42 [==============================] - 1s 19ms/step - loss: 0.1882 - accuracy: 0.9591 - val_loss: 1.5836 - val_accuracy: 0.7333
Epoch 31/50
42/42 [==============================] - 1s 19ms/step - loss: 0.1561 - accuracy: 0.9583 - val_loss: 1.2789 - val_accuracy: 0.7000
Epoch 32/50
42/42 [==============================] - 1s 19ms/step - loss: 0.1406 - accuracy: 0.9561 - val_loss: 1.6081 - val_accuracy: 0.7000
Epoch 33/50
42/42 [==============================] - 1s 19ms/step - loss: 0.2359 - accuracy: 0.9464 - val_loss: 1.1645 - val_accuracy: 0.7333
Epoch 34/50
42/42 [==============================] - 1s 19ms/step - loss: 0.1021 - accuracy: 0.9695 - val_loss: 1.7657 - val_accuracy: 0.6667
Epoch 35/50
42/42 

42/42 [==============================] - 1s 19ms/step - loss: 0.0938 - accuracy: 0.9702 - val_loss: 0.6473 - val_accuracy: 0.6667
Epoch 36/50
42/42 [==============================] - 1s 19ms/step - loss: 0.0901 - accuracy: 0.9725 - val_loss: 0.7028 - val_accuracy: 0.7667
Epoch 37/50
42/42 [==============================] - 1s 20ms/step - loss: 0.1001 - accuracy: 0.9665 - val_loss: 0.6205 - val_accuracy: 0.7333
Epoch 38/50
42/42 [==============================] - 1s 20ms/step - loss: 0.1380 - accuracy: 0.9717 - val_loss: 0.6438 - val_accuracy: 0.7000
Epoch 39/50
42/42 [==============================] - 1s 19ms/step - loss: 0.0888 - accuracy: 0.9769 - val_loss: 0.7399 - val_accuracy: 0.7333
Epoch 40/50
42/42 [==============================] - 1s 19ms/step - loss: 0.1007 - accuracy: 0.9643 - val_loss: 0.6638 - val_accuracy: 0.7667
Epoch 41/50
42/42 [==============================] - 1s 19ms/step - loss: 0.1324 - accuracy: 0.9688 - val_loss: 0.5336 - val_accuracy: 0.7667
Epoch 42/50
42/42 

In [43]:
print(all_accuracy)
import pandas as pd 
df = pd.DataFrame(all_accuracy)
df.to_csv('test.csv')

[0.7777777910232544, 0.8055555820465088, 0.8888888955116272, 0.6388888955116272, 0.8611111044883728, 0.8055555820465088, 0.7777777910232544, 0.5, 0.7222222089767456, 0.75, 0.7333333492279053, 0.7777777910232544, 0.7333333492279053, 0.6111111044883728, 0.6388888955116272, 0.8055555820465088, 0.6111111044883728, 0.5277777910232544, 0.5555555820465088, 0.5, 0.5277777910232544, 0.7777777910232544, 0.8611111044883728, 0.7222222089767456, 0.7777777910232544, 0.6388888955116272, 0.5833333134651184, 0.8333333134651184, 0.75, 0.8888888955116272, 0.5833333134651184, 0.7222222089767456, 0.6111111044883728, 0.6666666865348816, 0.7222222089767456, 0.8055555820465088, 0.7666666507720947, 0.6666666865348816, 0.6333333253860474]


In [140]:
import datetime
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
history = model.fit(x_train, y_train, epochs = 50, validation_data= (x_test, y_test),callbacks=[tensorboard_callback])
score, acc = model.evaluate(x_test, y_test)

Epoch 1/50
39/39 [==============================] - 2s 58ms/step - loss: 0.7017 - accuracy: 0.5858 - val_loss: 0.5693 - val_accuracy: 0.7319
Epoch 2/50
39/39 [==============================] - 1s 34ms/step - loss: 0.6533 - accuracy: 0.6788 - val_loss: 0.5839 - val_accuracy: 0.7391
Epoch 3/50
39/39 [==============================] - 1s 34ms/step - loss: 0.6793 - accuracy: 0.7087 - val_loss: 0.4724 - val_accuracy: 0.7536
Epoch 4/50
39/39 [==============================] - 1s 35ms/step - loss: 0.5993 - accuracy: 0.7193 - val_loss: 0.4537 - val_accuracy: 0.7754
Epoch 5/50
39/39 [==============================] - 1s 35ms/step - loss: 0.6023 - accuracy: 0.7751 - val_loss: 0.4327 - val_accuracy: 0.7899
Epoch 6/50
39/39 [==============================] - 1s 34ms/step - loss: 0.5284 - accuracy: 0.7896 - val_loss: 0.5186 - val_accuracy: 0.7899
Epoch 7/50
39/39 [==============================] - 1s 34ms/step - loss: 0.4642 - accuracy: 0.8091 - val_loss: 0.5392 - val_accuracy: 0.7464
Epoch 8/50
39

In [ ]:
### leave one out 


In [60]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [62]:
# %tensorboard --logdir logs

#### CNN 

In [141]:
## reshape 
x_train = np.reshape(x_train, (x_train.shape[0],new_x.shape[1],1))
x_test = np.reshape(x_test, (x_test.shape[0],new_x.shape[1],1))

In [142]:
print(x_train.shape,x_test.shape)
print(y_train.shape,y_test.shape)

(1236, 2048, 1) (138, 2048, 1)
(1236, 1) (138, 1)


In [143]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.wrappers import TimeDistributed
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import AveragePooling2D
from keras.layers.recurrent import LSTM
from keras import regularizers
from keras.callbacks import ModelCheckpoint
from keras.layers.wrappers import TimeDistributed
model = models.Sequential()
model.add(layers.Conv1D(16, 10, activation='relu',input_shape=(2048,1)))
model.add(layers.Conv1D(16, 3, activation='relu'))
model.add(layers.MaxPool1D(2))
model.add(layers.Flatten())

model.add(layers.Dense(100,activation="relu"))
model.add(layers.Dense(3,activation="softmax"))

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print(model.summary())
# input_shape = (trial_length, nchan, 1)
# l1 = 0
# model = Sequential()
# model.add(Conv2D(40, (30, 1), activation="relu", kernel_regularizer=regularizers.l1(l1), padding="same", input_shape=(2048,1)))
# model.add(Conv2D(40, (1, nchan), activation="relu", kernel_regularizer=regularizers.l1(l1), padding="valid"))
# model.add(AveragePooling2D((30, 1), strides=(15, 1)))
# model.add(Flatten())
# model.add(Dense(80, activation="relu"))
# model.add(Dense(2, activation="softmax"))
# model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["acc"])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 2039, 16)          176       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 2037, 16)          784       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 1018, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 16288)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               1628900   
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 1,630,163
Trainable params: 1,630,163
Non-trainable params: 0
____________________________________________

In [145]:
import datetime
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
history = model.fit(x_train, y_train, epochs = 30, validation_data= (x_test, y_test),callbacks=[tensorboard_callback])
score, acc = model.evaluate(x_test, y_test)

Epoch 1/30
39/39 [==============================] - 1s 23ms/step - loss: 0.2049 - accuracy: 0.9231 - val_loss: 0.6175 - val_accuracy: 0.7319
Epoch 2/30
39/39 [==============================] - 1s 22ms/step - loss: 0.1816 - accuracy: 0.9304 - val_loss: 0.6375 - val_accuracy: 0.7319
Epoch 3/30
39/39 [==============================] - 1s 22ms/step - loss: 0.1442 - accuracy: 0.9628 - val_loss: 0.7088 - val_accuracy: 0.7536
Epoch 4/30
39/39 [==============================] - 1s 23ms/step - loss: 0.1179 - accuracy: 0.9636 - val_loss: 0.7885 - val_accuracy: 0.7464
Epoch 5/30
39/39 [==============================] - 1s 22ms/step - loss: 0.1055 - accuracy: 0.9741 - val_loss: 0.9229 - val_accuracy: 0.7536
Epoch 6/30
39/39 [==============================] - 1s 22ms/step - loss: 0.1149 - accuracy: 0.9668 - val_loss: 0.8137 - val_accuracy: 0.7681
Epoch 7/30
39/39 [==============================] - 1s 22ms/step - loss: 0.1165 - accuracy: 0.9604 - val_loss: 0.8468 - val_accuracy: 0.7319
Epoch 8/30
39

#### CRNN

In [96]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.wrappers import TimeDistributed
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import AveragePooling2D
from keras.layers.recurrent import LSTM
from keras import regularizers
from keras.callbacks import ModelCheckpoint
from keras.layers.wrappers import TimeDistributed

model = models.Sequential()
model.add(layers.Conv1D(16, 10, activation='relu',input_shape=(2048,1)))
model.add(layers.Conv1D(16, 3, activation='relu'))
model.add(layers.MaxPool1D(2))
# model.add(AveragePooling2D((5, 1), strides=(5, 1)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(40, activation="sigmoid", dropout=0.25, return_sequences=False))
model.add(layers.Dense(100,activation="relu"))
model.add(layers.Dense(2,activation="softmax"))
## rmsprop
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print(model.summary())
# input_shape = (trial_length, nchan, 1)
# l1 = 0
# model = Sequential()
# model.add(Conv2D(40, (30, 1), activation="relu", kernel_regularizer=regularizers.l1(l1), padding="same", input_shape=(2048,1)))
# model.add(Conv2D(40, (1, nchan), activation="relu", kernel_regularizer=regularizers.l1(l1), padding="valid"))
# model.add(AveragePooling2D((30, 1), strides=(15, 1)))
# model.add(Flatten())
# model.add(Dense(80, activation="relu"))
# model.add(Dense(2, activation="softmax"))
# model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["acc"])

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_12 (Conv1D)           (None, 2039, 16)          176       
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 2037, 16)          784       
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 1018, 16)          0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 1018, 16)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 40)                9120      
_________________________________________________________________
dense_9 (Dense)              (None, 100)               4100      
_________________________________________________________________
dense_10 (Dense)             (None, 2)               

In [97]:
import datetime
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
history = model.fit(x_train, y_train, epochs = 20, validation_data= (x_test, y_test),callbacks=[tensorboard_callback])
score, acc = model.evaluate(x_test, y_test)

Epoch 1/20
40/40 [==============================] - 22s 562ms/step - loss: 0.6987 - accuracy: 0.4545 - val_loss: 0.6895 - val_accuracy: 0.5461
Epoch 2/20
40/40 [==============================] - 20s 493ms/step - loss: 0.6962 - accuracy: 0.5028 - val_loss: 0.6881 - val_accuracy: 0.5461
Epoch 3/20
40/40 [==============================] - 18s 438ms/step - loss: 0.6938 - accuracy: 0.5028 - val_loss: 0.6887 - val_accuracy: 0.5745
Epoch 4/20
40/40 [==============================] - 18s 442ms/step - loss: 0.6892 - accuracy: 0.5368 - val_loss: 0.6769 - val_accuracy: 0.5532
Epoch 5/20
40/40 [==============================] - 17s 435ms/step - loss: 0.6792 - accuracy: 0.5558 - val_loss: 0.6721 - val_accuracy: 0.5248
Epoch 6/20
40/40 [==============================] - 17s 436ms/step - loss: 0.6562 - accuracy: 0.6144 - val_loss: 0.6300 - val_accuracy: 0.6738
Epoch 7/20
40/40 [==============================] - 17s 435ms/step - loss: 0.6445 - accuracy: 0.6207 - val_loss: 0.6263 - val_accuracy: 0.6383